In [1]:
import pandas as pd
import numpy as np
#from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import classification_report,confusion_matrix
#from sklearn.cross_validation import KFold
#from sklearn.metrics import mean_absolute_error

# Read train and test files

train = pd.read_csv('input/train.csv')
#Drop the first column 'id' since it just has serial numbers. Not useful in the prediction process.
#X_train = X_train.iloc[:,1:]
X_train = train.drop(['loss', 'id'], 1)

test = pd.read_csv('input/test.csv')
ids = test['id']
X_test = test.drop(['id'], 1)
y_train = train['loss']

# Size of the dataframe

print(X_train.shape)
print(X_test.shape)

del train
del test

# We can see that there are 188318 training instances having 130 attributes

(188318, 130)
(125546, 130)


In [2]:
#cat1 to cat116 have strings. The ML algorithms we are going to study require numberical data
#One-hot encoding converts an attribute to a binary vector
split=116

#Variable to hold the list of variables for an attribute in the train and test data
labels = []

#get the names of all the columns
cols=X_train.columns

print(cols)

for i in range(0,split):
    train = X_train[cols[i]].unique()
    test = X_test[cols[i]].unique()
    labels.append(list(set(train) | set(test)))    

del train
del test

#Import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#One hot encode all categorical attributes
cats = []
for i in range(0, split):
    #Label encode
    label_encoder = LabelEncoder()
    label_encoder.fit(labels[i])
    feature = label_encoder.transform(X_train.iloc[:,i])
    feature = feature.reshape(X_train.shape[0], 1)
    #One hot encode
    onehot_encoder = OneHotEncoder(sparse=False,n_values=len(labels[i]))
    feature = onehot_encoder.fit_transform(feature)
    cats.append(feature)


# Make a 2D array from a list of 1D arrays
# Commented due to not enough memory error
#encoded_cats = np.column_stack(cats)

# Print the shape of the encoded data
#print(encoded_cats.shape)

# Concatenate encoded attributes with continuous attributes
#X_train_encoded = np.concatenate((encoded_cats,X_train.iloc[:,split:].values),axis=1)
X_train_encoded = np.concatenate((np.column_stack(cats),X_train.iloc[:,split:].values),axis=1)

print(X_train_encoded.shape)

del cats
del feature
del X_train
#del encoded_cats

Index(['cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9',
       'cat10',
       ...
       'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11',
       'cont12', 'cont13', 'cont14'],
      dtype='object', length=130)
(188318, 1190)


In [3]:
# Fit only to the training data
scaler = StandardScaler()
scaler.fit(X_train_encoded)
StandardScaler(copy=True, with_mean=True, with_std=True)

# Now apply the transformations to the data:
X_train = scaler.transform(X_train_encoded)

In [4]:
#X_train = X_train_encoded
#del X_train_encoded

In [ ]:
# One hot encoding also for X_test
cats = []
for i in range(0, split):
    #Label encode
    label_encoder = LabelEncoder()
    label_encoder.fit(labels[i])
    feature = label_encoder.transform(X_test.iloc[:,i])
    feature = feature.reshape(X_test.shape[0], 1)
    #One hot encode
    onehot_encoder = OneHotEncoder(sparse=False,n_values=len(labels[i]))
    feature = onehot_encoder.fit_transform(feature)
    cats.append(feature)

X_test_encoded = np.concatenate((np.column_stack(cats),X_test.iloc[:,split:].values),axis=1)

del cats
del feature
del X_test
print(X_test_encoded.shape)

X_test = scaler.transform(X_test_encoded)
#X_test = X_test_encoded
#del X_test_encoded

(125546, 1190)


In [ ]:
Y_train = np.asarray(y_train) 
print(Y_train.shape)

(188318,)


In [ ]:
#MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
#       beta_2=0.999, early_stopping=False, epsilon=1e-08,
#       hidden_layer_sizes=(130), learning_rate='constant',
#       learning_rate_init=0.001, max_iter=200, momentum=0.9,
#       nesterovs_momentum=True, power_t=0.5, random_state=None,
#       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
#       verbose=False, warm_start=False)

mlp = MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(130), learning_rate='constant',
       learning_rate_init=0.001, max_iter=300, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.001, validation_fraction=0.1,
       verbose=True, warm_start=False)

In [37]:
mlp.fit(X_train, Y_train)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=130, learning_rate='constant',
       learning_rate_init=0.01, max_iter=100, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.001, validation_fraction=0.1,
       verbose=10, warm_start=False)

In [38]:
predictions = mlp.predict(X_test)
print(mlp.score(X_train, Y_train))
#print(confusion_matrix(y_test,predictions))
#print(classification_report(y_test,predictions))
len(mlp.coefs_)
len(mlp.coefs_[0])
len(mlp.intercepts_[0])

0.739154993227


130

In [39]:
submission = pd.DataFrame()
submission['id'] = ids  
submission['loss'] = predictions
submission.to_csv('predictions.csv', index=False)